In [1]:
import logging

import numpy

from cicada.additive import AdditiveProtocolSuite
from cicada.communicator import SocketCommunicator
from cicada.encoding import Boolean
from cicada.logging import Logger

logging.basicConfig(level=logging.INFO)


def main(communicator):
    log = Logger(logging.getLogger(), communicator)
    protocol = AdditiveProtocolSuite(communicator)

    a = numpy.array([1, -2, 3, -4.5]) if communicator.rank == 0 else None
    b = numpy.array([1, 2, 3, -4.5]) if communicator.rank == 1 else None
    
    log.info(f"Player {communicator.rank} secret: {a}", src=0)
    log.info(f"Player {communicator.rank} secret: {b}", src=1)

    a_share = protocol.share(src=0, secret=a, shape=(4,))
    b_share = protocol.share(src=1, secret=b, shape=(4,))
    equal_share = protocol.equal(a_share, b_share)
    equal = protocol.reveal(equal_share, encoding=Boolean())
    
    log.info(f"Player {communicator.rank} result: {equal}")

SocketCommunicator.run(world_size=3, fn=main);

INFO:root:Player 0 secret: [ 1.  -2.   3.  -4.5]
INFO:root:Player 1 secret: [ 1.   2.   3.  -4.5]
INFO:root:Player 0 result: [ True False  True  True]
INFO:root:Player 1 result: [ True False  True  True]
INFO:root:Player 2 result: [ True False  True  True]
